# 1. YOLO란?

YOLO(You Only Look Once, 욜로족 할 때 욜로 아닙니다ㅜ)라는 모델은 

실시간 객체탐지를 위한 시스템으로,

이미지 내의 객체를 식별하고 위치를 추정하는 데 주로 사용됩니다.

실시간 비디오 분석, 자율주행차량, 보안시스템 등 다양한 분야에서 널리 사용되며,

버전을 거듭하면서 원래 빨랐던 속도는 갈수록 더 빨라지고,

(단점으로 자주 거론되었던) 정확도 측면에서도 지속적으로 개선되어

사실상 지금은 "느리지만 정교하다"는 모델들과 비교해도 

정확도 면에서 큰 차이가 나지 않습니다.

우리가 이번에 사용할 버전은 ultralytics라는 업체에서 발표한 YOLOv8입니다.

가장 기본적인 사용법은, 

아래처럼 선행학습된(pretrained) 모델을 가져다가 

바로 예측하는 것입니다. 

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO("yolov8n.pt")  # n은 nano의 약자로, 가장 경량입니다. nsmlx 순으로 큰 모델입니다.
results = model.predict("https://ultralytics.com/images/bus.jpg", save=True)
# model.predict(img) 대신 model(img)로도 동일하게 실행가능


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\1_object_detect_img_and_video\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 204.1ms
Speed: 5.3ms preprocess, 204.1ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\detect\predict


출력되는 메시지들을 모두 꼼꼼히 읽어야 하는 건 아니지만

중요한 정보가 들어있기도 합니다. 예를 들어 위에서 출력된 메시지를

간단히 읽어보면

Downloading https://ultralytics.com/images/bus.jpg to 'bus.jpg'...

100%|██████████| 476k/476k [00:00<00:00, 13.9MB/s]

image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\1_object_detection\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 145.9ms
Speed: 6.8ms preprocess, 145.9ms inference, 10.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\Users\Administrator\PycharmProjects\yolo_project\runs\detect\predict4

---

"이미지 안에서 4명의 사람, 1대의 버스, 1개의 정지표지판을 검출했다"

"이미지 해상도는 640x480,
전처리에 6.8밀리초, 예측에 약 0.15초, 후처리에 10.6밀리초가 소요되었다."

"결과 이미지는 ./runs/detect/predict6에 저장되었다."

등의 정보가 출력되었습니다.

기존 파일과 결과물을 한 번 비교해볼까요?

기존 이미지는

![](./bus.jpg)


결과물은

![](https://i.ibb.co/N6w220n/bus.jpg)

입니다.

이미지를 보면, 각각의 박스 위에 클래스명과 일종의 확률이 표시되어 있습니다.

(이 확률은 "신뢰점수confidence score"라고도 부릅니다.)

예를 들어 좌측 베이지색 털코트를 입은 남자의 0.87은 

YOLO 모델이 87%의 확률로 사람일 거라고 예측했다는 의미입니다.

여기까지 YOLO 모델을 통해 

기본적인 object detection 실습을 해보았습니다.

여러분이 가지고 계신 이미지를 가지고

한 번 모델에 넣고 예측해보시기 바랍니다.

저는...

우리 사전미팅 때 이미지 일부를 가지고 테스트해봤습니다.


![](./zoom_capture.jpg)

In [4]:
results = model.predict("./zoom_capture.jpg", save=True)


image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\1_object_detect_img_and_video\zoom_capture.jpg: 384x640 6 persons, 1 laptop, 151.4ms
Speed: 4.5ms preprocess, 151.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\detect\predict


![](https://i.ibb.co/dLb2KZ3/zoom-capture.jpg)


저를 포함해서 화면상의 6명이 전부 검출되었네요. 

(오정민 샘은 코 밑으로 잘려서 검출이 안 될 수도 있겠다 싶었거든요.)

재미있는 건, 좌측의 코드화면을 YOLO가 "노트북"이라고 

50% 이상의 확률로 예측했다는 점인데요. 

학습할 때 노트북의 외관보다는 모니터에 초점을 두고 학습했나보구나 싶네요.


# 2. object detection만 가능한가?

YOLO에는 기본적으로 네 종류의 작업이 제공됩니다.

## (1) Object Detection

이미지상의 객체 종류(class)와 위치를 리턴해 줍니다.

리턴 정보와 opencv, matplotlib 등을 이용하여 이미지상에 박스와 클래스를 표시할 수 있지만

YOLO에서는 기본적으로 위 정보가 표시된 이미지도 저장할 수 있습니다. (save=True)

## (2) Segmentation 

클래스별로 픽셀 단위의 영역을 레이블링할 수 있습니다.

ZOOM에서 사람을 제외하면 배경 부분에 가상배경을 표시한다든지, 블러 처리하는 데에도

세그멘테이션(분할) 작업이 수행됩니다.

"yolov8n-seg.pt" 등의 모델을 불러오기만 하면 됩니다.

In [5]:
model = YOLO("yolov8n-obb.pt")
results = model.predict("https://ultralytics.com/images/bus.jpg", save=True)


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\1_object_detect_img_and_video\bus.jpg: 640x480 4 persons, 1 bus, 1 skateboard, 204.6ms
Speed: 5.6ms preprocess, 204.6ms inference, 18.7ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\segment\predict


![](https://i.ibb.co/0FYFGCm/bus.jpg)

잘 보시면 아시겠지만 "사람"이라고 해서 모두 하나로 묶어버리는 것이 아니라

같은 클래스라도 구분해서 영역을 추론해냅니다.

![](https://i.ibb.co/sRwt8rF/196.png)

> 이를 전문용어로는 "instance segmentation"이라고 합니다.
> instance segmentation 외에도
> 같은 클래스의 객체는 하나로 묶는 작업 : semantic segmentation
> 셀 수 있거나(사람, 자동차) 셀 수 없는(하늘, 땅) 객체 모두를 구분 : panoptic segmentation
> 등이 있습니다.

![](https://i.ibb.co/qB9gj0s/195.png)

우리가 사용하는 yolov8n-seg.pt는 인스턴스세그멘테이션을 수행하고 있습니다.



## (3) Pose Estimation

이미지 내에서 person에 해당하는 객체의 특징을 추출해서

점과 선으로 연결된 17개의 뼈대 좌표를 추출(및 표시)해줍니다.

화면안에 있는 사람 기준으로 차례대로 각각

0 : "코",
1 : "오른쪽 눈",
2 : "왼쪽 눈",
3 : "오른쪽 귀",
4 : "왼쪽 귀",
5 : "오른쪽 어깨",
6 : "왼쪽 어깨",
7 : "오른쪽 팔꿈치",
8 : "왼쪽 팔꿈치",
9 : "오른쪽 손목",
10 : "왼쪽 손목",
11 : "오른쪽 골반",
12 : "왼쪽 골반",
13 : "오른쪽 무릎",
14 : "왼쪽 무릎",
15 : "오른쪽 발",
16 : "왼쪽 발",

입니다.

예시화면
![](https://i.ibb.co/RbxF8Wj/197.png)

최근 헬스 및 교정 관련 앱에 활발하게 도입되고 있습니다.
예시를 들어보면

![](https://i.ibb.co/YtH22vz/286830010-cf016a41-589f-420f-8a8c-2cc8174a16de.gif) 
(인정사정이 없기 때문에 반만 내려갔다 올라오면 여지없이 카운트를 안 합니다..)

![](https://i.ibb.co/5x49pgn/286833598-cb20f316-fac2-4330-8445-dcf5ffebe329.gif)
(체력측정은 로봇이 대체해버릴 수 있겠지요?)

운동 관련 솔루션은 기본적으로

푸쉬업pushup, 턱걸이pullup 및 윗몸일으키기abworkout 등
 
세 가지가 기본제공되고 있습니다.

노드의 상호작용만 정의할 수 있다면

스쿼트나 데드리프트, 벤치프레스 등 다양한 운동의

운동량을 측정할 수 있는 모델을 만들어볼 수 있습니다. 

## (4) 분류(Classification)

마지막 모델은 바로 "분류"입니다.

이미지상의 주인공(처럼 보이는) 오브젝트가 

"어떤 클래스"에 가까운 사진인지

각 클래스일 확률을 리턴합니다. 

(욜로 분류모델은 imagenet-pretrained model이므로, 레이블이 1,000종입니다.) 

두 개 이상의 객체가 있더라도 대표객체 하나에 대해서만 추론을 합니다. (아래 사진)

다만, 분류를 수행할 수 있는 모델은 굳이 YOLO가 아니어도

단순하고 빠르면서 성능도 뛰어난 모델이 많이 때문에

분류작업을 굳이 YOLO로 수행하는 일은 드문 편이지만,


In [6]:
model = YOLO("yolov8n-cls.pt")
results = model.predict("https://ultralytics.com/images/bus.jpg", save=True)


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 C:\Users\Administrator\PycharmProjects\jss_tutorials\1_object_detect_img_and_video\bus.jpg: 224x224 minibus 0.54, police_van 0.24, trolleybus 0.06, golfcart 0.02, streetcar 0.02, 28.9ms
Speed: 10.3ms preprocess, 28.9ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)
Results saved to C:\Users\Administrator\PycharmProjects\jss_tutorials\runs\classify\predict2


![](https://i.ibb.co/cD9KPhb/bus.jpg)


이렇게 YOLOv8의 각 모델의 개요에 대해 알아보았습니다.
